* [Import required packages](#section-one)
* [Define model](#section-two)
* [Extract features from image dataset](#section-three)

<a id="section-one"></a>
# Import required packages

In [1]:
import pickle as pk
import os
import keras as keras
import keras.applications.vgg16 as vgg
import keras.preprocessing.image as img
import string
import torch
import tensorflow as tf
from tqdm import tqdm

<a id="section-two"></a>
# Define a model

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg16_model = vgg.VGG16()
model = keras.Model(inputs = vgg16_model.inputs, outputs = vgg16_model.layers[-2].output)
print(model.summary())

2022-01-03 22:01:31.138434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 22:01:31.139456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 22:01:31.140098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 22:01:31.140827: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

553476096/553467096 [==============================] - 10s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

<a id="section-three"></a>
# Extract features from the image dataset

In [3]:
image_to_features = {}
dataset_path = '../input/flickr8kdataset/Flickr8k_Dataset/Flicker8k_Dataset'
for file_name in tqdm(os.listdir(dataset_path)):
    filename = os.path.join(dataset_path, file_name)
    image = img.load_img(filename, target_size=(224, 224))
    image = img.img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = vgg.preprocess_input(image)
    feature = model.predict(image, verbose=0)
    image_to_features[file_name.split('.')[0]] = feature

  0%|          | 0/8091 [00:00<?, ?it/s]2022-01-03 22:01:47.315414: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-03 22:01:48.218118: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
100%|██████████| 8091/8091 [07:53<00:00, 17.08it/s]


In [4]:
len(image_to_features)

8091

In [5]:
pk.dump(image_to_features, open('features.pkl', 'wb'))